# PINT Example notebook

The PINT "homepage":  https://github.com/nanograv/PINT

PINT will normally be run via a script, or for "work", via an interactive session like ipython or a frontend

## Times of Arrival (TOAs)

Read in some TOAs.

Note:  The first time that things get read in, lots of processing (can) happen.  Can take a few seconds.  But a "pickle" file is saved, so the next import (if nothing changed), things are much faster (typically ~1 second)

In [ ]:
import pint.toa as toa
#t = toa.get_TOAs("examples/NGC6440E.tim")
t = toa.get_TOAs("examples/J0613-sim.tim")

In [ ]:
t.print_summary()

In [ ]:
t.get_mjds()[0]

Vast majority of the information is in an [Astropy Table](http://astropy.readthedocs.org/en/latest/table/):

In [ ]:
t.table.colnames

Lots of cool things that tables can do...

In [ ]:
tt = t.table
#tt.show_in_browser()

Can do fancy sorting, selecting, re-arranging very easily.

In [ ]:
select = tt['error'] < 20
print(select)

In [ ]:
tt['tdb'][select]

Many PINT routines / classes / functions use [Astropy Units](http://astropy.readthedocs.org/en/latest/units/) internally or externally:

In [ ]:
import astropy.units as u
t.get_errors() < 20 * u.ns

The times in each row contain (or are derived from) [Astropy Time](http://astropy.readthedocs.org/en/latest/time/) objects:

In [ ]:
t0 = tt['mjd'][0]

In [ ]:
t0.tai

But the most useful timescale, TDB (and maybe soon others) are also stored as long double (i.e. "float96") Numpy arrays:

In [ ]:
tt['tdbld'][:3]

## Timing (or other) Models

Now let's define and load a timing model

In [ ]:
import pint.models as models
#m = models.get_model('examples/NGC6440E.par')
m = models.get_model('examples/J0613-sim.par')

In [ ]:
print(m.as_parfile())

In [ ]:
print m.RAJ.value, m.F0.value, m.T0.value

Timing models are basically composed of "delay" terms and "phase" terms:

In [ ]:
m.delay_funcs

In [ ]:
m.phase_funcs

Can easily show/compute individual terms...

In [ ]:
ds = m.solar_system_shapiro_delay(tt)
print(ds)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(t.get_mjds(high_precision=False), ds*1e6, 'x')
plt.xlabel("MJD") ; plt.ylabel("Delay ($\mu$s)")

or all of the terms added together:

In [ ]:
m.delay(tt)

In [ ]:
m.phase(tt)

## Residuals

In [ ]:
import pint.residuals as r

In [ ]:
rs = r.resids(t, m).phase_resids

In [ ]:
plt.plot(t.get_mjds(high_precision=False), rs, 'x')
plt.title("%s Pre-Fit Timing Residuals" % m.PSR.value)
plt.xlabel('MJD'); plt.ylabel('Residual (phase)')
plt.grid()


## Fitting and Post-Fit residuals

The fitter is *completely* separate from the model and the TOA code.  So you can use any type of fitter with some easy coding.  This example uses a very simple Powell minimizer from the SciPy optimize module. 

In [ ]:
import pint.fitter as fit
f = fit.wls_fitter(t, m)

In [ ]:
f.get_fitparams()

In [ ]:
f.call_minimize()

In [ ]:
print "Best fit has reduced chi^2 of", f.resids.chi2_reduced
print "RMS in phase is", f.resids.phase_resids.std()
print "RMS in time is", f.resids.time_resids.std().to(u.us)
print "\n Best model is:"
print f.model.as_parfile()


In [ ]:
plt.errorbar(t.get_mjds(high_precision=False),
             f.resids.time_resids.to(u.us).value,
             t.get_errors().to(u.us).value, fmt='x')
plt.title("%s Post-Fit Timing Residuals" % m.PSR.value)
plt.xlabel('MJD'); plt.ylabel('Residual (us)')
plt.grid()

## Other interesting things

We can make Barycentered TOAs in a single line!

In [ ]:
tt['tdbld'] - m.delay(tt) / 86400.0

In [ ]:
import pint.toa as toa, pint.models as models
t = toa.get_TOAs("examples/J0613-sim.tim")
m = models.get_model('examples/J0613-sim.par')

In [ ]:
t.get_mjds(high_precision=False) - m.delay(t.table)

In [ ]:
m.BT_delay(t.table)